<!-- ABSTRACT -->
Put this notebook in root directory!

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

from thesis_results.utils import set_plot_style, aid_mapping, color_mapping
set_plot_style()

In [ ]:
data = json.load(open("data/ausgrid/group_4.json", "r"))

In [ ]:
stats = {aid : {"demand": np.array(data[aid_mapping[aid]]["demand"]).reshape(1096,24),
                "pv": np.array(data[aid_mapping[aid]]["pv"]).reshape(1096,24),
                "load": np.array(data[aid_mapping[aid]]["demand"]).reshape(1096,24) - np.array(data[aid_mapping[aid]]["pv"]).reshape(1096,24)
                } for aid in aid_mapping.keys()}

stats["total"] = {"demand": sum([stats[aid]["demand"] for aid in aid_mapping.keys()]),
                  "pv": sum([stats[aid]["pv"] for aid in aid_mapping.keys() if "prosumer" in aid])}

stats["total"]["load"] = stats["total"]["demand"] - stats["total"]["pv"]

### Table 3.4

In [ ]:
for aid in sorted(list(aid_mapping.keys()) + ["total"]):

    mean_demand = stats[aid]["demand"].sum(axis=1).mean()
    mean_pv = stats[aid]["pv"].sum(axis=1).mean()

    std_demand = stats[aid]["demand"].sum(axis=1).std()
    std_pv = stats[aid]["pv"].sum(axis=1).std()

    if "consumer" in aid:
        print(f"{aid} & {mean_demand:.2f} $\\pm$ {std_demand:.2f} & --- \\\\")
    else:
        print(f"{aid} & {mean_demand:.2f} $\\pm$ {std_demand:.2f} & {mean_pv:.2f} $\\pm$ {std_pv:.2f} \\\\")

### Load Plots (3.2, 3.3, 3.4)

In [ ]:
def plot_hourly_demand(data_dict,
                       data_key,
                       filter,
                       agents,
                       color_mapping,
                       plot_variance=False,
                       legend_loc=0,
                       save_path=None):
    
    hours = np.arange(24)  # 0–23
    
    plt.figure()
    
    for agent in agents:
        
        values = data_dict[agent][data_key]  # shape (1096, 24)

        if filter is not None:
            values = values[filter, :]

        mean_vals = values.mean(axis=0)
        std_vals = values.std(axis=0)
        
        color = color_mapping.get(agent, None)
        
        # Line plot
        # plt.plot(hours, mean_vals, label=agent, color=color, linewidth=2)
        plt.plot(hours, mean_vals, label="Mean ± Std. Dev.", color=color, linewidth=2) # Hack for Community
        
        if plot_variance:
            # Shaded std
            plt.fill_between(hours,
                            mean_vals - std_vals,
                            mean_vals + std_vals,
                            color=color,
                            alpha=0.2)
    
    # Formatting
    plt.xticks(hours, [f"{h}" for h in hours])   # 00–23 labels
    plt.xlabel("Hour of Day ($t$)")
    plt.ylabel("Inflexible Load (kWh)")
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.legend(loc=legend_loc)
    plt.tight_layout()

    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")

In [ ]:
summer_filter = [x and (not y) for x, y in zip(data['meta']['is_summer'], data['meta']['is_weekend'])]
winter_filter = [x and (not y) for x, y in zip(data['meta']['is_winter'], data['meta']['is_weekend'])]
weekend_filter = [x or (not (y or z)) for x, y, z in zip(data['meta']['is_weekend'], data['meta']['is_summer'], data['meta']['is_winter'])]

In [ ]:
plot_hourly_demand(data_dict=stats,
                   data_key="load",
                   filter=weekend_filter,
                   agents=["total"],
                   color_mapping=color_mapping,
                   plot_variance=True,
                   legend_loc=0,
                   save_path="thesis_results/plots/03_dataset/weekend_load_ec.png")

### ToU Plot (3.1)

In [ ]:
# Grid Config, Time of Use
ToU = [0.15, 0.22, 0.44]

FiT = 0.04  # Grid Config, Feed-in Tariff

# Timestep (hours) to ToU period, AER 2019
summer_timemap = [0, 0, 0, 0, 0, 0, 0, # 12 AM - 7 AM
                        1, 1, 1, 1, 1, 1, 1, # 7 AM - 2 PM
                        2, 2, 2, 2, 2, 2, # 2 PM - 8 PM
                        1, 1, # 8 PM - 10 PM
                        0, 0] # 10 PM - 12 AM

winter_timemap = [0, 0, 0, 0, 0, 0, 0, # 12 AM - 7 AM
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, # 7 AM - 5 PM
                        2, 2, 2, 2, # 5 PM - 9 PM
                        1, # 9 PM - 10 PM
                        0, 0] # 10 PM - 12 AM

weekend_timemap = [0, 0, 0, 0, 0, 0, 0, # 12 AM - 7 AM
                   1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, # 7 AM - 10 PM
                   0, 0] # 10 PM - 12 AM

In [ ]:
price_map = {
    "Summer": [ToU[i] for i in summer_timemap],
    "Winter": [ToU[i] for i in winter_timemap],
    "Weekend + Other Months": [ToU[i] for i in weekend_timemap],
    "FiT": [FiT for _ in range(24)]
}

color_mapping = {
    "Summer": "#d62728",
    "Winter": "#1f77b4",
    "Weekend + Other Months": "#ff7f0e",
    "FiT": "#2ca02c"
}

In [ ]:
hours = np.arange(24)  # 0–23

plt.figure()

for pricing in ["Weekend + Other Months", "Winter", "Summer", "FiT"]:
    
    values = price_map[pricing]
    
    # Line plot
    plt.plot(hours, values, label=pricing, color=color_mapping[pricing], linewidth=2)

# Formatting
plt.xticks(hours, [f"{h}" for h in hours])   # 00–23 labels
plt.xlabel("Hour of Day ($t$)")
plt.ylabel("Price in $/kWh")
plt.grid(True, which="both", linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()